In [ ]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=746f75fce68c9719588c444750391fd707a8f082b89faf3f94807bd8763c3fba
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


#### Realizando todos os imports e fazendo a requisição, confirmando sucesso da requisição com o status_code retornado.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
from datetime import datetime
import pandasql

# Requisitando dados da página de eventos por data
r = requests.get('https://nuforc.org/webreports/ndxevent.html')
r.status_code

200

#### Testando algumas funcionalidades da biblioteca "requests".

In [ ]:
r.encoding

'ISO-8859-1'

In [ ]:
r.text

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN"><HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1"><META NAME="GENERATOR" CONTENT="Mozilla/4.5b2 [en] (WinNT; I) [Netscape]"><TITLE>UFO Report Index by Month</TITLE></HEAD><BODY><FONT FACE="Calibri"><FONT SIZE=+1>National UFO ReportingCenter</FONT></FONT><BR><FONT FACE="Calibri"><FONT SIZE=+1>UFO Report Index by Month</FONT><BR><P><FONT FACE="Calibri"><FONT SIZE=+1><A HREF="https://www.nuforc.org">NUFORC Home</A><BR><BR></FONT></BODY><TABLE  CELLSPACING=1><THEAD><TR><TH BGCOLOR=#c0c0c0 BORDERCOLOR=#000000 ><FONT style=FONT-SIZE:11pt FACE="Calibri" COLOR=#000000>Reports</TH><TH BGCOLOR=#c0c0c0 BORDERCOLOR=#000000 ><FONT style=FONT-SIZE:11pt FACE="Calibri" COLOR=#000000>Count</TH></TR></THEAD><TBODY><TR VALIGN=TOP><TD><A HREF= ndxe202206.html>06/2022</A></TD><TD>267</TD></TR>\n<TR VALIGN=TOP><TD><A HREF= ndxe202205.html>05/2022</A></TD><TD>396</TD></TR>\n<TR VALIGN=TOP><TD><A HREF= ndxe202204.

#### Pegando o texto da requisição e formatando conforme HTML através da biblioteca "BeautifulSoup".

In [ ]:
# Transforma o documento em um objeto "BeautifulSoup" que 'encapsula
soup = BeautifulSoup(r.text, 'html.parser')

#### Testando função find_all para montagem de vetores do dataFrame.

In [ ]:
soup.find_all('a')

[<a href="https://www.nuforc.org">NUFORC Home</a>,
 <a href="ndxe202206.html">06/2022</a>,
 <a href="ndxe202205.html">05/2022</a>,
 <a href="ndxe202204.html">04/2022</a>,
 <a href="ndxe202203.html">03/2022</a>,
 <a href="ndxe202202.html">02/2022</a>,
 <a href="ndxe202201.html">01/2022</a>,
 <a href="ndxe202112.html">12/2021</a>,
 <a href="ndxe202111.html">11/2021</a>,
 <a href="ndxe202110.html">10/2021</a>,
 <a href="ndxe202109.html">09/2021</a>,
 <a href="ndxe202108.html">08/2021</a>,
 <a href="ndxe202107.html">07/2021</a>,
 <a href="ndxe202106.html">06/2021</a>,
 <a href="ndxe202105.html">05/2021</a>,
 <a href="ndxe202104.html">04/2021</a>,
 <a href="ndxe202103.html">03/2021</a>,
 <a href="ndxe202102.html">02/2021</a>,
 <a href="ndxe202101.html">01/2021</a>,
 <a href="ndxe202012.html">12/2020</a>,
 <a href="ndxe202011.html">11/2020</a>,
 <a href="ndxe202010.html">10/2020</a>,
 <a href="ndxe202009.html">09/2020</a>,
 <a href="ndxe202008.html">08/2020</a>,
 <a href="ndxe202007.html">07

#### Montando vetores que serão passados como parâmetros para criação do dataframe e posteriormente do arquivo .csv, constando nestes a filtragem solicitada na etapa 1.1 (09/1997 -> 08/2017).


In [ ]:
data = []

for index, link in enumerate(soup.find_all('a')):
    if index != 0 and link.text != 'UNSPECIFIED / APPROXIMATE':
      aux = datetime.strptime(link.text, '%m/%Y').strftime('%m/%Y')
      data.append(aux)

In [ ]:
contagem = []

for index, link in enumerate(soup.find_all('td')):
    if (index % 2 != 0) and link.text != '376':
      contagem.append(int(link.text))

In [ ]:
links = []

for index, link in enumerate(soup.find_all('a')):
    if index != 0 and link.get('href') != 'ndxe.html':
      links.append(str('https://nuforc.org/webreports/' + link.get('href')))

#### Função de criação do dataframe "geral" a partir dos vetores criados anteriormente.

In [ ]:
def create_dataframe(data, contagem, links): 
  d = {
    "Data": data,
    "Contagem": contagem,
    "Links": links
  }

  df = DataFrame(d)

  return df

#### Criando dataframe utilizando a função e passando o mesmo como parâmetro para criação do .csv.

In [ ]:
df = create_dataframe(data, contagem, links)
df.to_csv("ovnis.csv", index = False)

In [ ]:
df

,Data,Contagem,Links
0,08/2017,421,https://nuforc.org/webreports/ndxe201708.html
1,07/2017,528,https://nuforc.org/webreports/ndxe201707.html
2,06/2017,430,https://nuforc.org/webreports/ndxe201706.html
3,05/2017,369,https://nuforc.org/webreports/ndxe201705.html
4,04/2017,423,https://nuforc.org/webreports/ndxe201704.html
...,...,...,...
235,01/1998,85,https://nuforc.org/webreports/ndxe199801.html
236,12/1997,76,https://nuforc.org/webreports/ndxe199712.html
237,11/1997,132,https://nuforc.org/webreports/ndxe199711.html
238,10/1997,116,https://nuforc.org/webreports/ndxe199710.html


#### Nesta parte, a partir do arquivo "ovnis.csv" foi montado um outro: "dados-filtrados.csv", constando neste o intervalo de datas solicitado (para essa parte, foi utilizada a linguagem JavaScript). Além disso, foi feito um array constando tabelas vindas do redirecionamento da coluna "Links" no dataFrame filtrado (dados-filtrados.csv).

In [ ]:
df = pd.read_csv('./dados-filtrados.csv')
array = df.values.tolist()

tabelas = []

for val in array:
    tabelas.append(pd.read_html(val[2]))

len(tabelas)

240

#### Aqui foi montado um array para dinamizar o acesso às diversas tabelas e preparar os dados para estudo com consultas SQL.

In [ ]:
frames = []

for val in tabelas:
  frames.append(val[0])
  
frames_concatenados = pd.concat(frames)

frames_concatenados.to_csv('todos_ovnis.csv', sep=',')

#### Realizando consultas solicitadas na etapa 1.2 do projeto.

In [ ]:
# Saber a quantidade de linhas, observações ou variáveis que foram coletadas.
# RESPOSTA: 100398	

q1 = pd.read_csv('todos_ovnis.csv')

q = """
SELECT * FROM q1;
"""

pandasql.sqldf(q, locals())

,Unnamed: 0,Date / Time,City,State,Country,Shape,Duration,Summary,Posted,Images
0,0,8/31/17 22:00,Norwalk,CT,USA,Light,Extremely brief,Bright green light zig-zagged in the sky and d...,9/5/17,None
1,1,8/31/17 22:00,Elizabeth,WV,USA,Light,13 seconds,((HOAX??)) Looked like a star that moved acros...,9/5/17,None
2,2,8/31/17 21:00,San Diego,CA,USA,Rectangle,30 seconds,Rectangle four white lights two red flashing l...,9/5/17,None
3,3,8/31/17 20:15,E. Rio Vista,CA,USA,Light,20 seconds,I'm a truck driver headed E on Hwy 12 just W o...,9/5/17,None
4,4,8/31/17 19:30,Magna,UT,USA,Sphere,30 minutes,"Bright glowing, reflected surface. Sphere-like...",9/5/17,None
...,...,...,...,...,...,...,...,...,...,...
100393,56,9/2/97 15:00,Zurich (Switzerland),None,Switzerland,Disk,unknown,fast moving object merging from in back of swi...,1/28/99,None
100394,57,9/1/97 23:00,Albany,ME,USA,Light,2 min,"Star makes right angle turns, speeds away.",2/1/07,None
100395,58,9/1/97 22:30,Carlsbad,CA,USA,Light,3-5 min,Round orb of light seen in back yard.between a...,12/12/09,None
100396,59,9/1/97 20:00,Pleasant View,TN,USA,Fireball,5 seconds,large white fireball or super flare...,10/31/03,None


In [ ]:
# Quantos relatos ocorreram por estado em ordem decrescente?

q2 = pd.read_csv('todos_ovnis.csv')

q = """
SELECT COUNT(state), state FROM q2 GROUP BY state HAVING COUNT(state) > 1 ORDER BY count(state) DESC; 
"""

pandasql.sqldf(q, locals())

,count(state),State
0,11470,CA
1,5618,FL
2,4925,WA
3,4181,TX
4,3899,NY
...,...,...
63,21,YT
64,20,PE
65,18,PR
66,5,YK


In [ ]:
# Remover possíveis campos vazios (sem estado).
# RESPOSTA: 7242

q3 = pd.read_csv('todos_ovnis.csv')

q = """
SELECT * FROM q3 WHERE state is null OR state = "";
"""

pandasql.sqldf(q, locals())

,Unnamed: 0,Date / Time,City,State,Country,Shape,Duration,Summary,Posted,Images
0,58,8/26/17 00:00,Mackay (Australia),None,Australia,Triangle,5,Small cloaked (camouflaged) triangle UFO with ...,9/5/17,None
1,63,8/25/17 22:00,Barcelona (Spain),None,Spain,Light,3 seconds,"Observed a small white flash in the night sky,...",9/5/17,None
2,75,8/25/17 18:30,Ovid,None,USA,Oval,15 seconds,A metallic oval with bright white light in and...,9/5/17,None
3,112,8/22/17 18:30,Zejtun (Malta),None,Malta,Flash,1 seconds,Please see the video and check second object.,8/24/17,None
4,127,8/21/17 13:30,Sidmouth (UK/England),None,United Kingdom,Sphere,15 minutes,Two stationary spheres seen off of Devon coast...,9/19/19,None
...,...,...,...,...,...,...,...,...,...,...
7237,81,10/11/97 23:30,"Eslö, Scania (Sweden)",None,Sweden,None,10 minutes,During an IYCW (International Youth Camp Weeke...,1/28/99,None
7238,82,10/11/97 22:00,Hafnarfjordur (Iceland),None,Iceland,Sphere,5 min,playing with a jet,6/12/08,None
7239,23,9/18/97 22:15,Doha (Qatar),None,Qatar,Circle,10-15 sec,UFO sighting In a Gulf Desert,8/5/01,None
7240,48,9/10/97 16:00,Lierskogen (Norway),None,Norway,Unknown,3 minutes,It looked as a pointed horseshoe. It was full ...,4/26/99,None


In [ ]:
# Limitar a análise aos estados dos Estados Unidos.

q4 = pd.read_csv('todos_ovnis.csv')

q = """
SELECT * FROM q4 WHERE Country = "USA"; 
"""

pandasql.sqldf(q, locals()).head()

,Unnamed: 0,Date / Time,City,State,Country,Shape,Duration,Summary,Posted,Images
0,0,8/31/17 22:00,Norwalk,CT,USA,Light,Extremely brief,Bright green light zig-zagged in the sky and d...,9/5/17,None
1,1,8/31/17 22:00,Elizabeth,WV,USA,Light,13 seconds,((HOAX??)) Looked like a star that moved acros...,9/5/17,None
2,2,8/31/17 21:00,San Diego,CA,USA,Rectangle,30 seconds,Rectangle four white lights two red flashing l...,9/5/17,None
3,3,8/31/17 20:15,E. Rio Vista,CA,USA,Light,20 seconds,I'm a truck driver headed E on Hwy 12 just W o...,9/5/17,None
4,4,8/31/17 19:30,Magna,UT,USA,Sphere,30 minutes,"Bright glowing, reflected surface. Sphere-like...",9/5/17,None


In [ ]:
# Consulta por cidades, com o objetivo de saber quais contêm o maior número de relatos (cidades que apresentem ao menos 10 relatos).

q5 = pd.read_csv('todos_ovnis.csv')

q = """
SELECT COUNT(city), city FROM q5 GROUP BY city HAVING COUNT(city) > 10 ORDER BY count(city) DESC; 
"""

pandasql.sqldf(q, locals())


,count(city),City
0,561,Phoenix
1,550,Seattle
2,482,Portland
3,476,Las Vegas
4,396,San Diego
...,...,...
1799,11,Wrightsville Beach
1800,11,Xenia
1801,11,Yorktown
1802,11,Zeeland


In [ ]:
# Fazer uma query exclusiva para o estado com maior número de relatos, buscando cidades que possuam um número superior a 10 relatórios. Enfatizar a cidade, a quantidade de relatos e formato do objeto não identificado.
# Primeiro descobrindo estado com maior número de relatos (CA)

q7_1 = pd.read_csv('todos_ovnis.csv')

q = """
SELECT COUNT(state), state FROM q7_1 GROUP BY state ORDER BY COUNT(state) DESC;
"""

pandasql.sqldf(q, locals())



,COUNT(state),State
0,11470,CA
1,5618,FL
2,4925,WA
3,4181,TX
4,3899,NY
...,...,...
81,1,South Carolina
82,1,Southern District
83,1,VI
84,1,"Washington, DC"


In [ ]:
# Fazer uma query exclusiva para o estado com maior número de relatos, buscando cidades que possuam um número superior a 10 relatórios. Enfatizar a cidade, a quantidade de relatos e formato do objeto não identificado.
# Agora fazendo a query baseado no estado com maior número de relatos

q7_2 = pd.read_csv('todos_ovnis.csv')

q = """
SELECT COUNT(city) AS "Number of Reports", state, city, shape FROM q7_2 WHERE state = "CA" GROUP BY city HAVING COUNT(city)> 10 ORDER BY COUNT(city) DESC;
"""

pandasql.sqldf(q, locals())

,Number of Reports,State,City,Shape
0,392,CA,San Diego,Light
1,382,CA,Los Angeles,Triangle
2,244,CA,Sacramento,light
3,224,CA,San Jose,Triangle
4,197,CA,San Francisco,Triangle
...,...,...,...,...
250,11,CA,Nipomo,Light
251,11,CA,Pismo Beach,Triangle
252,11,CA,Seal Beach,Light
253,11,CA,West Los Angeles,Sphere
